In [1]:
import itertools
import json
import math
import os
import random
import re
import sys
from collections import Counter
from datetime import datetime

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import utils
from tqdm import tqdm

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

7. 学习日志表-activity_learn_log

---

In [2]:
activity_learn_log = pd.read_csv('./data/activity_learn_log.csv')

导出学习记录

In [3]:
json_data = {}

In [4]:
json_data['userId_list'] = sorted([int(x) for x in activity_learn_log['userId'].unique()])

In [5]:
def calc(df: pd.DataFrame):
    dicts = utils.count_pair(list(zip(df["mediaType"], df["learnedTime"])))
    return {"watching": dicts.get("video", 0), "doing": sum(dicts.values()) - dicts.get("video", 0)}

In [6]:
json_data['user_learning_type'] = activity_learn_log[["userId", "mediaType", "learnedTime"]].groupby("userId").apply(calc).to_dict()

In [7]:
json_data["user_learning_record"] = (
    activity_learn_log[["userId", "learnedTime"]].groupby("userId").agg(list).to_dict()["learnedTime"]
)

In [8]:
json.dump(json_data, open("./out/个性化课程推荐.json", "w"))

In [9]:
activity_learn_log.groupby('mediaType')['learnedTime'].sum()

mediaType
doc            1924650
download      11655739
exercise        989711
homework       3831024
ppt               2016
testpaper     76173014
text           2021533
video        567950648
Name: learnedTime, dtype: int64

In [10]:
# dicts = {}
(activity_learn_log.groupby('mediaType')['learnedTime'].sum() / activity_learn_log.groupby('mediaType')['learnedTime'].sum().sum())

mediaType
doc          0.002896
download     0.017539
exercise     0.001489
homework     0.005765
ppt          0.000003
testpaper    0.114624
text         0.003042
video        0.854642
Name: learnedTime, dtype: float64

In [11]:
np.array(sorted(activity_learn_log['userId'].unique()))

array([     3,      5,     13, ..., 174592, 174600, 174601], dtype=int64)

In [12]:
activity_learn_log['mediaType'].value_counts()

video        10856853
testpaper     1264731
download       225334
homework        66453
text            37673
doc             28196
exercise        16115
ppt               111
Name: mediaType, dtype: int64

In [13]:
activity_learn_log['userId'].value_counts()

167157    31477
5183      29031
159863    27987
145060    22993
164177    20639
          ...  
167688        1
165006        1
163253        1
29116         1
163193        1
Name: userId, Length: 10095, dtype: int64

In [14]:
activity_learn_log2 = activity_learn_log[activity_learn_log['event'].isin({"doing", "watching"})]
len(activity_learn_log2)

11492514

In [15]:
activity_learn_log.iloc[2]['data']

'{"lastTime":"1679989451","lastLearnTime":"505.51","taskId":"263255","event":"doing","learnedTime":48}'

In [16]:
activity_learn_log.iloc[2]

id                                                              3
activityId                                                 263272
courseTaskId                                               263255
userId                                                     162609
mediaType                                                   video
event                                                       doing
data            {"lastTime":"1679989451","lastLearnTime":"505....
learnedTime                                                    48
createdTime                                            1679989499
Name: 2, dtype: object

8. 试卷信息表-testpaper

---

In [17]:
testpaper = pd.read_csv('./data/testpaper.csv')

In [18]:
testpaper = testpaper[(testpaper['type'] == 'testpaper') & (testpaper['status'] != 'draft') ]

9. 试卷结果表-testpaper_result

---

In [19]:
testpaper_result = pd.read_csv('./data/testpaper_result.csv')

C:\Users\Tanphoon\AppData\Local\Temp\ipykernel_3652\4110415055.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  testpaper_result = pd.read_csv('./data/testpaper_result.csv')


In [20]:
testpaper_result = testpaper_result[testpaper_result['type'] == 'testpaper']
testpaper_result = testpaper_result[['userId', 'paperName', 'courseId', 'score']]

融合表8和表9

In [21]:
# 求交集
cap = set(testpaper_result['courseId'].unique()) & set(testpaper['courseId'].unique())
# 表8数据
total_score_ =  testpaper[testpaper['courseId'].isin(cap)][["courseId", "name", "score"]].set_index(["courseId", "name"]) 
# 表9数据
total_score = testpaper_result.groupby(['courseId','paperName' ])['score'].max()
# 合并
for idx, score in total_score_.itertuples():
    if idx in total_score:
        total_score[idx] = score

In [22]:
testpaper_result

,userId,paperName,courseId,score
4,3,试卷2,2,2.0
5,3,试卷2,2,0.0
1438,15422,Python数据分析方向 - 试题,425,10.0
1439,15422,Python数据分析,406,16.0
1440,15422,Python数据分析,406,28.0
...,...,...,...,...
349031,93923,计算机程序设计员-四级理论考试模拟卷②,11628,63.0
349032,169949,计算机程序设计员-四级理论考试模拟卷①,11628,87.0
349033,174507,泰迪智能双创工作室知识考核,12778,0.0
349034,169255,计算机程序设计员-四级理论考试模拟卷⑤,11628,83.0


In [23]:
user_data = testpaper_result.groupby(['userId', 'courseId', 'paperName']).max()
user_data

score
userId courseId paperName                    
3      2        试卷2                       2.0
       14       《Hadoop大数据基础与应用》期中考试试卷    0.0
       96       课程题目模板V1.2（上传）            0.0
       263      提交项目成果                    0.0
       338      《Hadoop大数据基础与应用》期中考试试卷    0.0
...                                       ...
174581 11109    人工智能（高级）正式卷               0.0
174582 11109    人工智能（高级）正式卷               0.0
174583 11109    人工智能（高级）正式卷               0.0
174584 11109    人工智能（高级）正式卷               0.0
174585 11109    人工智能（高级）正式卷               0.0

[145789 rows x 1 columns]

In [24]:
def get_max_score(row):
    idx = (row.name[1], row.name[2])
    return total_score[idx] if total_score[idx] != 0 else 1
user_data['rating'] =  user_data.apply(lambda x:int(x['score'] / get_max_score(x) * 10), axis=1)

In [25]:
user_data = user_data.groupby(["userId", "courseId"]).max().reset_index()

In [26]:

class UserItemRecommond:
    def __init__(self, user_data: dict) -> None:
        self.user_data = user_data
        self.user_similarity_matrix = {}
        self.user_related_matrix = {}
        self.average_rating = {}
        self.item_user = {}

        print("\033[0;32mBuilding item-user table...\033[0m")
        for user, items in tqdm(user_data.items()):
            for x in items:
                self.item_user.setdefault(x, [])
                self.item_user[x].append(user)
        print(f"len(item) = {len(self.item_user)}, len(user) = {len(self.user_data)}")

        print("\033[0;32mBuild user common-rating item matrix...\033[0m")
        for item, users in tqdm(self.item_user.items()):
            for u in users:
                self.user_similarity_matrix.setdefault(u, {})
                self.user_related_matrix.setdefault(u, {})
                for v in users:
                    if u == v:
                        continue
                    self.user_similarity_matrix[u].setdefault(v, 0)
                    self.user_related_matrix[u].setdefault(v, [])
                    self.user_similarity_matrix[u][v] += 1
                    self.user_related_matrix[u][v].append(item)
        for user, items in user_data.items():
            self.average_rating[user] = np.mean(list(items.values()))

    def calc_user_sim(self):
        print("\033[0;32mCalculating user similarity matrix...\033[0m")
        r = self.user_data
        # for u, related_users in self.user_related_matrix.items():
        for u, related_users in tqdm(self.user_related_matrix.items()):
            # 获取目标用户评价电影的数量
            for v, items in related_users.items():
                pearson1 = 0
                pearson2 = 0
                pearson3 = 0
                W1 = 0
                W2 = 0
                for x in items:
                    pearson1 += (
                        math.exp(-math.log(len(self.item_user[x]), 10))
                        * (r[u][x] - self.average_rating[u])
                        * (r[v][x] - self.average_rating[v])
                    )
                    pearson2 += pow(r[u][x] - self.average_rating[u], 2)
                    pearson3 += pow(r[v][x] - self.average_rating[v], 2)
                    W1 += r[u][x]
                    W2 += r[v][x]
                if pearson1 == 0:
                    self.user_similarity_matrix[u][v] = 0
                else:
                    W = math.sqrt(W1 / sum(r[u].values()) * W1 / sum(r[v].values()))
                    IS = len(self.user_related_matrix[u][v]) / (
                        len(r[u]) + len(r[v]) - len(self.user_related_matrix[u][v])
                    )
                    pearson = pearson1 / (math.sqrt(pearson2) * math.sqrt(pearson3))
                    self.user_similarity_matrix[u][v] = IS * W * pearson

    # 针对目标用户u，找到其最相似的K个相似用户，产生N个推荐
    def recommend(self, u: int, K: int, N: int):
        r = self.user_data
        predict = {}
        userSimSum = 0
        # 寻找u的最近邻居集K个，v表示相似用户，sim相似度
        for v, sim in sorted(self.user_similarity_matrix[u].items(), key=lambda x: x[1], reverse=True)[0:K]:
            userSimSum += abs(sim)
            for item, rating in r[v].items():
                if item in r[u]:
                    continue
                predict.setdefault(item, 0)
                predict[item] += sim * (rating - self.average_rating[v])
        for item in predict.keys():
            predict[item] = float(self.average_rating[u] + predict[item] / userSimSum)
            predict[item] = float(f"{predict[item]:.2f}")
        recommond_courseId = sorted(predict.items(), key=lambda x: x[1], reverse=True)[0:N]
        return recommond_courseId
    def hot_top(self, N: int):
        return list(dict(sorted(self.item_user.items(), key=lambda x : len(x[1]), reverse=True)).keys())[0:N]

In [27]:
user_data

,userId,courseId,score,rating
0,3,2,2.0,10
1,3,14,0.0,0
2,3,96,0.0,0
3,3,263,0.0,0
4,3,338,0.0,0
...,...,...,...,...
47369,174581,11109,0.0,0
47370,174582,11109,0.0,0
47371,174583,11109,0.0,0
47372,174584,11109,0.0,0


In [28]:
user_score = user_data.groupby('userId').apply(lambda x: x.set_index('courseId')['score'].to_dict()).to_dict()
user_rating = user_data.groupby('userId').apply(lambda x: x.set_index('courseId')['rating'].to_dict()).to_dict()

In [29]:
AR_CF_score = UserItemRecommond(user_score)

Building item-user table...


100%|██████████| 31172/31172 [00:00<00:00, 1416904.30it/s]


len(item) = 1415, len(user) = 31172
Build user common-rating item matrix...


100%|██████████| 1415/1415 [00:03<00:00, 425.64it/s]


In [30]:
json_data = {}

In [31]:
userId_list = sorted([int(x) for x in testpaper_result['userId'].unique()])
json_data['userId_list'] = userId_list

In [32]:
json_data['courseId_by_score'] = {user: dict(AR_CF_score.recommend(user, 10, 10)) for user in userId_list}

In [33]:
json_data['hot_courseId_by_score'] = AR_CF_score.hot_top(10)

In [34]:
AR_CF_rating = UserItemRecommond(user_rating)

Building item-user table...


100%|██████████| 31172/31172 [00:00<00:00, 1246792.01it/s]


len(item) = 1415, len(user) = 31172
Build user common-rating item matrix...


100%|██████████| 1415/1415 [00:02<00:00, 554.15it/s] 


In [35]:
json_data['courseId_by_rating'] = {user: dict(AR_CF_rating.recommend(user, 10, 10)) for user in userId_list}

In [36]:
json_data['hot_courseId_by_rating'] = AR_CF_rating.hot_top(10)

In [37]:
json.dump(json_data, open('./out/个性化课程推荐.json', "w"))